# Advent of Code

The docstrings are pretty bad.

In [15]:
import os
import re
import math
import string
import copy
from datetime import datetime
from typing import List, Tuple, Optional, Any, Callable, Dict, Set
import textwrap
from collections import defaultdict, Counter, deque
from pprint import pprint

from pydantic.dataclasses import dataclass, Field
import requests as req
import pandas as pd
import numpy as np
import numpy.ma as ma
import altair as alt

## Helper Functions

In [2]:
def print_solutions(a1sol, a2sol):
    print(textwrap.dedent(f"""
        a1_solution: {a1sol}
        a2_solution: {a2sol}
    """))

---
## Advent Day 8

In [3]:
# This was a nightmare to try to do without brute-force and with trying to keep to 
# "General" patterns.  In truth, this can be done using a few if-else statements.
# I wanted to practice a bit of my magic methods and the like, so I made some
# objects which turned out to be of fairly limited use, but were fun to make.

ORIG_DIGIT_ENCODINGS = list(map(set, ["abcefg", "cf", "acdeg",  
"acdfg", "bcdf", "abdfe", 
"abdefg", "acf", "abcdefg", 
"abcdfg"]))

class Digit:
    def __init__(self, name: str, segments: str):
        self.name = name
        self.segments = set(segments)
        self.length = len(self.segments)
        
    def __repr__(self):
        return f"Digit({self.name}, {self.segments}, {self.length})"
        
    def __and__(self, other):
        return self.segments.intersection(other.segments)
    
    def __lt__(self, other):
        self.name < other.name
        
    def __lte__(self, other):
        self.name <= other.name
    
def create_digits() -> List[Digit]:
    return [Digit(0, "abcefg"), Digit(1, "cf"), Digit(2, "acdeg"),  
            Digit(3, "acdfg"), Digit(4, "bcdf"), Digit(5, "abdfe"), 
            Digit(6, "abdefg"), Digit(7, "acf"), Digit(8, "abcdefg"), 
            Digit(9, "abcdfg")]

def parse_code_string(code_string: str) -> Tuple[Set, Set]:
    """Parses input of aoc_8."""
    code_string = code_string.split(" | ")
    code, output = code_string

    codes = set(list(map(lambda x: "".join(sorted(x)), code.split(" "))))
    output = list(map(lambda x: "".join(sorted(x)), output.split(" ")))

    return (codes, output)

def determine_1478(codes: Set) -> Tuple[Dict[str, Digit], Dict[str, Digit]]:
    possible_digits = {"".join(sorted(code)): sorted([digit for digit in digits if digit.length == len(code)]) 
                       for code in codes}
    
    matching_matches = {digit: possibilities[0].name
                       for digit, possibilities in possible_digits.items()
                       if len(possibilities) == 1}
    return matching_matches, possible_digits

class DigitIntersectionSignature:
    def __init__(self, signature: str):
        self.signature = signature
        self.intersection_signature = [
            len(signature.intersection(ORIG_DIGIT_ENCODINGS[i]))
            for i in [1, 4, 7, 8]]
        
def decode_by_intersection_signature(code_output):
    c, o = code_output
    matches = determine_1478(c)[0]
    matches_by_number = {v: set(k) for k, v in matches.items()}

    digit_signatures = [DigitIntersectionSignature(sig) for sig in ORIG_DIGIT_ENCODINGS]
    data = np.array(list(digit_signature.intersection_signature for n, digit_signature in enumerate(digit_signatures)))

    encoded_values = defaultdict(int)
    for k in c:
        signature = [len(set(k).intersection(matches_by_number[i])) for i in [1, 4, 7, 8]]
        encoded_values[k] = [n for n, row in enumerate(data) if (np.array(row) == signature).all()][0]

    output_vals = [encoded_values[val] for val in o]
    return int("".join(map(str, output_vals)))

digits = create_digits()

# sample = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
# edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
# fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
# fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
# aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
# fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
# dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
# bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
# egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
# gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

with open("a08.csv", "r") as sample_f:
    sample = sample_f.read()
    
sample_split = sample.split("\n")
code_output_list = [parse_code_string(row) for row in sample_split]

def aoc_8_a() -> int:
    """Solves aoc_8a."""
    values_1478 = defaultdict(int)
    for codes, output in code_output_list:
        matches = determine_1478(codes)[0]
        for item in output:
            if item in matches:
                values_1478[matches[item]] += 1


    return sum(values_1478.values())

def aoc_8_b() -> int:
    return sum(decode_by_intersection_signature(code_output) for code_output in code_output_list)

print_solutions(aoc_8_a(), aoc_8_b())


a1_solution: 488
a2_solution: 1040429



---
## Advent Day 9

In [31]:
data_raw = """
2199943210
3987894921
9856789892
8767896789
9899965678
""".strip().split("\n")

with open("a09.csv", "r") as f:
    data_raw = f.read().strip().split("\n")

data_raw = np.array(list(map(lambda x: [int(y) for y in x], data_raw)))

class SmokeMap:
    def __init__(self, chart: np.ndarray):
        self.chart = chart
        self.n_rows, self.n_cols = self.chart.shape
        
        #!! This needs to be cleared every time.
        # TODO: How do I deal with something like this?
        self.basin_chart = (self.chart.copy() != 9).astype(int)
        
        
    def find_neighbors(self, idx: int, jdx: int) -> List[int]:
        """Finds up-down-left-right neighbors of (idx, jdx)."""
        neighbor_indices = [
            [idx + 1, jdx],
            [idx - 1, jdx],
            [idx, jdx + 1],
            [idx, jdx - 1]
        ]
        
        valid_neighbor_indices = [
            coord for coord in neighbor_indices
            if coord[0] >= 0 and coord[0] < self.n_rows
            and coord[1] >= 0 and coord[1] < self.n_cols
        ]
        
        neighbors = [self.chart[nbidx[0], nbidx[1]] for nbidx in valid_neighbor_indices]
        return neighbors, valid_neighbor_indices

    def test_if_local_min(self, idx: int, jdx: int) -> bool:
        """Compares neighbors to value of (idx, jdx), sees if
        (idx, jdx) is strictly less then all of them."""
        val = self.chart[idx, jdx]
        neighbors = self.find_neighbors(idx, jdx)[0]
        for neighbor in neighbors:
            if neighbor <= val:
                return False
        return True

    def calculate_total_risk_level(self):
        """Gets all local minima, adds one to its value, sums the results."""
        risk_level = 0
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if self.test_if_local_min(row, col):
                    risk_level += self.chart[row, col] + 1
                    
        return risk_level
    
    
    def find_basin_size(self, idx: int, jdx: int) -> List[Tuple[int, int]]:
        """Finds basin around (idx, jdx) as defined 
        in https://adventofcode.com/2021/day/9#part2."""
        if self.basin_chart[idx, jdx] == 0:
            return 0
        
        basin_size = 1
        self.basin_chart[idx, jdx] = 0
        neighbors = [nbr for nbr in self.find_neighbors(idx, jdx)[1]
                     if self.basin_chart[nbr[0], nbr[1]]]

        for neighbor in neighbors:    
            basin_size += self.find_basin_size(neighbor[0], neighbor[1])

        return basin_size
    
    def calculate_basin_sizes(self):
        """Gets basin sizes."""
        basin_sizes = []
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if size := self.find_basin_size(row, col):
                     basin_sizes.append(size)
                    
        return basin_sizes   

sm = SmokeMap(data_raw)
print_solutions(
    sm.calculate_total_risk_level(), 
    math.prod(sorted(sm.calculate_basin_sizes(), reverse=True)[:3])
)


a1_solution: 478
a2_solution: 1327014



---
## Adent Day 10

In [58]:
CHUNK_DELIMS = [["{", "[", "<", "("], ["}", "]", ">", ")"]]
DELIMS_OPEN = dict(zip(CHUNK_DELIMS[1], CHUNK_DELIMS[0]))
DELIMS_CLOSE = dict(zip(CHUNK_DELIMS[0], CHUNK_DELIMS[1]))

class DelimString:
    
    def __init__(self, line: str):
        self.line = line
        self.stack = []
        
        self.is_incomplete = None
        self.is_corrupt = None
        self.first_illegal_character = None
        
        self._check_stack()
        
    def _clear_stack(self):
        self.stack = []
        
    def _check_stack(self):
        """Checks stack for corruption or incompleteness."""
        self._clear_stack()
        
        for symbol in self.line:
            # If symbol is an end delim, either we have it joining its opening
            # or it is misaligned.
            if symbol in ")}]>":
                if self.stack[-1] != DELIMS_OPEN[symbol]:
                    # print(f"Expected {delims_close[self.stack[-1]]} got {symbol}.")
                    self.is_corrupt = True
                    self.first_illegal_character = symbol
                    break
                else:
                    # Pop the corresponding opening delim.
                    self.stack.pop()
            else:
                # Otherwise, it's an open delim.
                self.stack.append(symbol)
        
        if self.is_corrupt is None:        
            self.is_incomplete = len(self.stack) != 0                 

In [84]:
with open("a10.csv", "r") as f:
    data = [line.strip() for line in f.readlines()]
    
points = {
    ")": 3,
    "]": 57,
    "}": 1197,
    ">": 25137
}

def aoc10_a():
    illegal_characters = []
    for line in data:
        ds = DelimString(line)
        if not ds.is_incomplete:
            if ds.is_corrupt:
                illegal_characters.append(points[ds.first_illegal_character])

    return sum(illegal_characters)

### Part 2.

COMPLETION_POINTS = {
    ")": 1,
    "]": 2,
    "}": 3,
    ">": 4
}

def find_completion(data: List[str]) -> List[str]:
    """Finds delims to complete string."""
    incomplete = []
    for line in data:
        ds = DelimString(line)
        if ds.is_incomplete:
            incomplete.append(ds)

    completions = []
    for ds in incomplete:
        completions.append([DELIMS_CLOSE[symbol] for symbol in ds.stack[::-1]])

    return completions
        
def calculate_completion_score(completion: List[str]) -> int:
    """Calculates completion score a la AoC10."""
    score = 0
    for symbol in completion:
        score *= 5
        score += COMPLETION_POINTS[symbol]

    return score     

def aoc10_b(data: List[str] = data) -> int:
    completions = find_completion(data)
    scores = []
    for completion in completions:
        scores.append(calculate_completion_score(completion))
    return sorted(scores)[int(len(scores) / 2)]

print_solutions(aoc10_a(), aoc10_b())


a1_solution: 436497
a2_solution: 2377613374

